In [1]:
#todo consider going through base text data before normal preprocessing
# and finding words which are conjoined & then separating them?
# is this computationally intractable?
# for each whitespace-separated token, go through each index in the token
# & see whether the substring up to that index and the substring after that index are both words?
# http://pyenchant.github.io/pyenchant/api/enchant.checker.html
# find spellcheck errors in text, then for each of those errors
# do the above iteration over char indices in the word to find a splitting point
# then if one's found you make a new string with the two actual words separated by
# a space and then modify the original text in place

In [2]:
from DataNamesReference import *
import enchant
from enchant.checker import SpellChecker
from UtilityFuncs import getNonEmptyLines

import time


engDict = enchant.DictWithPWL(tag=LANG_CODE, pwl= SPELL_CHECKER_PERSONAL_WORD_LIST_PATH)



benefitsLines = getNonEmptyLines(benefitsSummaryFilePath)


In [4]:
def unspliceWords(langDict, rawText):
    checker = SpellChecker(lang=langDict, text=rawText)
    allBadWordCount = 0
    splicedWords = []

    for errToken in checker:
        allBadWordCount += 1
        errStr = errToken.word
        errLen = len(errStr)

        for charInd in range(2, errLen-2):
            firstWordStr = errStr[0:charInd]
            secondWordStr = errStr[charInd:]
            if checker.check(firstWordStr) \
                and checker.check(secondWordStr):
                splicedWords.append((errStr, firstWordStr, secondWordStr))
                errToken.replace_always(firstWordStr + " " + secondWordStr)
                break
    
    unsplicedText = checker.get_text()
    return unsplicedText, allBadWordCount, splicedWords

testBenefitsRawText = " ".join(benefitsLines[:180])
testBenefitsRawTextWordLen = len(testBenefitsRawText.split())

benefitsTestStartTime = time.time()
testBenefitsUnsplicedText, totalNumBadWords, splicedWords = unspliceWords(engDict, testBenefitsRawText)
benefitsTestEndTime = time.time()
benefitsTestDuration = benefitsTestEndTime - benefitsTestStartTime

benefitsTestDurationPerWord = benefitsTestDuration/testBenefitsRawTextWordLen

print("to process %d words, %d of which were misspelled and %d of which were specifically "
      "a pair of words spliced together,\n the spell checker required %f seconds, for %f seconds per word processed"
      % (testBenefitsRawTextWordLen, totalNumBadWords, len(splicedWords), benefitsTestDuration,
         benefitsTestDurationPerWord))

testBenefitsWordsAdded = len(testBenefitsUnsplicedText.split()) - testBenefitsRawTextWordLen

bigTestBenefitsRawText = " ".join(benefitsLines[:1800])
bigTestBenefitsRawTextWordLen = len(bigTestBenefitsRawText.split())

benefitsBigTestStartTime = time.time()
bigTestBenefitsUnsplicedText, bigTestTotalNumBadWords, bigTestSplicedWords = unspliceWords(engDict, bigTestBenefitsRawText)
benefitsBigTestEndTime = time.time()
benefitsBigTestDuration = benefitsBigTestEndTime - benefitsBigTestStartTime

benefitsBigTestDurationPerWord = benefitsBigTestDuration/bigTestBenefitsRawTextWordLen

print("to process %d words, %d of which were misspelled and %d of which were specifically "
      "a pair of words spliced together,\n the spell checker required %f seconds, for %f seconds per word processed"
      % (bigTestBenefitsRawTextWordLen, bigTestTotalNumBadWords, len(bigTestSplicedWords), benefitsBigTestDuration,
         benefitsBigTestDurationPerWord))

bigTestBenefitsWordsAdded = len(bigTestBenefitsUnsplicedText.split()) - bigTestBenefitsRawTextWordLen


to process 4372 words, 422 of which were misspelled and 249 of which were specifically a pair of words spliced together,
 the spell checker required 4.601748 seconds, for 0.001053 seconds per word processed
to process 54800 words, 5434 of which were misspelled and 2496 of which were specifically a pair of words spliced together,
 the spell checker required 39.923631 seconds, for 0.000729 seconds per word processed
